<a href="https://colab.research.google.com/github/kooga0682/FAIRCUT-Trainer/blob/master/face2bmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd /content/drive/My Drive/Make/face2bmi-keras/

/content/drive/My Drive/Make/face2bmi-keras


In [0]:
# -*- coding: utf-8-unix -*-

import os, sys
sys.path.append("/face2bmi-keras/keras-vggface")

import glob
import pandas as pd
import numpy as np
import h5py
import re

# Keras: Tensorflow backend
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.utils import np_utils


def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def flush_progress_bar(barname, rate):
    number = int(rate/0.05)+1
    bar = ('#' * number).ljust(20, '-')
    
    sys.stdout.write(f"\r{barname} : [{bar}] {rate*100:.2f}%")

class BMIDataset:
    def __init__(self):
        self.image_shape = (224, 224, 3)
        self.dataset_store_file_path = "./stored_bmi_dataset/bmi_dataset.h5"
        
    def load_data(self, image_dir_path):
        if os.path.exists(self.dataset_store_file_path):
            print("Stored dataset found!\nLoading...")
            
            with h5py.File(self.dataset_store_file_path, 'r') as loaded_dataset:
                x_train = loaded_dataset["x_train"].value
                t_train = loaded_dataset["t_train"].value
                x_test = loaded_dataset["x_test"].value
                t_test = loaded_dataset["t_test"].value


        else:
            print("Stored dataset not found.\nMaking...")
            x = []
            t = []
        
            files = sorted(glob.glob(image_dir_path + "/*.bmp"), key=numericalSort)
            
            n = len(files)
            i = 0
            
            # Get image and bmi value from original dataset
            for imgfile in files:
                img = load_img(imgfile, target_size=(224,224) ) #(self.image_shape[:2]))
                img_array = img_to_array(img)
                filename = os.path.basename(imgfile)
                #bmi = self.filename2bmi(filename)
                bmiLevel = self.filename2bmiLevel(filename)
                
                flush_progress_bar("Loading image", i/n)
                
                x.append(img_array)
                t.append(bmiLevel)
        
                i += 1
        
            x = np.array(x)
            x = x.astype("float32") / 255.0
            t = np.array(t)
            t = t.astype("int32")
            t_categorical = np_utils.to_categorical(t, 9)

            x_train = x[:3368]
            x_test = x[3368:]
            t_train = t_categorical[:3368]
            t_test = t_categorical[3368:]
            
            # Store dataset to file
            if not os.path.exists(os.path.dirname(self.dataset_store_file_path)):
                os.mkdir(os.path.dirname(self.dataset_store_file_path))
            
            with h5py.File(self.dataset_store_file_path, "w") as store_dataset:
                store_dataset.create_dataset("x_train", data=x_train)
                store_dataset.create_dataset("t_train", data=t_train)
                store_dataset.create_dataset("x_test", data=x_test)
                store_dataset.create_dataset("t_test", data=t_test)
                store_dataset.flush()
    
        return((x_train, t_train), (x_test, t_test))

    def filename2bmiLevel(self, filename):    
        # Get index
        tempstr = filename.replace(".bmp", "")
        tempstrs = tempstr.split("_")
        index = int(tempstrs[1])

        # Get BMI from csv file by index
        df = pd.read_csv("./DataFromMIT-CSAIL/data.csv")
        
        bmi = df["bmi"][index]

        bmiLevel = 1

        if bmi < 16.0:
            bmiLevel = 8
        elif bmi >= 16.0 and bmi < 17.0:
            bmiLevel = 7
        elif bmi >= 17.0 and bmi < 18.5:
            bmiLevel = 6
        elif bmi >= 18.5 and bmi < 25.0:
            bmiLevel = 5
        elif bmi >= 25.0 and bmi < 30.0:
            bmiLevel = 4
        elif bmi >= 30.0 and bmi < 35.0:
            bmiLevel = 3
        elif bmi >= 35.0 and bmi < 40.0:
            bmiLevel = 2
        elif bmi >= 40.0:
            bmiLevel = 1
        else:
            bmiLevel = 5
            
        return bmiLevel
    
    
    def filename2bmi(self, filename):    
        # Get index
        tempstr = filename.replace(".bmp", "")
        tempstrs = tempstr.split("_")
        index = int(tempstrs[1])

        # Get BMI from csv file by index
        df = pd.read_csv("./DataFromMIT-CSAIL/data.csv")
        
        bmi = df["bmi"][index]
        
        return float(bmi)



def main():
    
    # Load dataset
    dataset = BMIDataset()
    ( (x_train, t_train), (x_test, t_test) ) = dataset.load_data("./DataFromMIT-CSAIL/Images")

    print('\n')
    
    print(t_test[:10])
    
    

if __name__ == "__main__":
    main()


Stored dataset found!
Loading...


[[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [0]:
# -*- coding: utf-8-unix -*-

import os, sys
sys.path.append("./keras-vggface")

import glob
import pandas as pd
import numpy as np
import h5py

from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras_vggface.vggface import VGGFace

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model



def network():
    #custom parameters
    nb_class = 9
    hidden_dim = 512
    learning_late = 0.01

    vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
    last_layer = vgg_model.get_layer('pool5').output
    x = Flatten(name='flatten')(last_layer)
    x = Dense(hidden_dim, activation='relu', name='fc6')(x)
    x = Dense(hidden_dim, activation='relu', name='fc7')(x)
    out = Dense(nb_class, activation='softmax', name='fc8')(x)
    custom_vgg_model = Model(vgg_model.input, out)
    vgg_model.summary()
    optimizer = SGD(lr=learning_late)
    custom_vgg_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["mae"])

    return custom_vgg_model


def main():
    # Parameters
    batch_size = 50
    epochs = 400
    verbose = 0 #1
    validation_split = 0.2
    
    # Load dataset
    dataset = BMIDataset()
    (x_train, t_train), (x_test, t_test) = dataset.load_data("./Data/Images")
    input_shape = dataset.image_shape
    
    model = network()    
    
    
    print(t_test)

    
    # Learn
    model.fit(x_train, t_train,
              batch_size=batch_size, epochs=epochs, callbacks=None, 
              verbose=verbose, validation_split=validation_split)
    model.save('saved_model.h5')
    
    # # Eval
    score = model.evaluate(x_test, t_test, verbose=verbose, batch_size=batch_size)
    print("\nTest loss:", score[0])
    print('Test mean abs error:', score[1])
    
    

if __name__ == "__main__":
    main()


Stored dataset found!
Loading...
58916864/58909280 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
____________________________________________

In [0]:
from keras.models import load_model

batch_size = 50
verbose = 1 #1
validation_split = 0.2
    
# Load dataset
dataset = BMIDataset()
(x_train, t_train), (x_test, t_test) = dataset.load_data("./Data/Images")
input_shape = dataset.image_shape

model = load_model('saved_model.h5')
score = model.evaluate(x_test, t_test, verbose=verbose, batch_size=batch_size)
print("\nTest loss:", score[0])
print('Test mean abs error:', score[1])

Stored dataset found!
Loading...


W0728 06:36:36.174724 140040920635264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 06:36:36.242967 140040920635264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 06:36:36.331175 140040920635264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0728 06:36:38.922859 140040920635264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0728 06:36:38.924287 140040920635264 deprecation_

838/838 [==============================] - 18s 21ms/step

Test loss: 5.587452828172852
Test mean abs error: 0.14455433463708883
